<a href="https://colab.research.google.com/github/dev-S-t/ML/blob/main/Pinecone_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
doc_paths = "/content/drive/MyDrive/mental health assignment/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install "unstructured[pdf]"


In [ ]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

In [2]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [3]:
loader = DirectoryLoader(doc_paths, glob="**/*.pdf")

In [4]:
docs = loader.load()

In [5]:
docs[0]

Document(metadata={'source': '/content/drive/MyDrive/mental health assignment/2024-Behavioral-Health-Resource-Guide-Final-6.27.24.pdf'}, page_content='2024 Resource Guide FOR MARYLAND’S MID-SHORE REGION\n\nPublic Behavioral Health Services for Caroline County Dorchester County Kent County Queen Anne’s County Talbot County\n\nMid Shore Behavioral Health, Inc. www.midshorebehavioralhealth.org 410-770-4801\n\nQuick Reference Resources\n\nMid Shore Behavioral Health, Inc. 2024 Resource Guide\n\n1\n\nTable of Contents\n\nQUICK REFERENCE RESOURCES .................................................................................................................................................. 1\n\nTABLE OF CONTENTS .................................................................................................................................................................. 2\n\nMID SHORE BEHAVIORAL HEALTH, INC. ................................................................................

In [6]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [13]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "mental-health"

# Split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)

In [14]:
split_docs[25]

Document(metadata={'source': '/content/drive/MyDrive/mental health assignment/2024-Behavioral-Health-Resource-Guide-Final-6.27.24.pdf'}, page_content="and Treatment\n\n\uf076 Psychiatric Assessment \uf076 Psychiatric Medication Services \uf076 Psychiatric Rehabilitation Program Off-\n\nSite and On-Site\n\n\uf076 Suboxone \uf076 Substance Use Education /Prevention \uf076 Substance Use Intensive Outpatient \uf076 Substance Use Outpatient \uf076 Telepsychiatry/ Telemental Health\n\n\uf076 Methadone\n\nMid Shore Behavioral Health, Inc. 2024 Resource Guide\n\n32\n\nRebirth Inc.\n\nAddress 225 N Division St. Salisbury MD 21801\n\nPayment Accepted \uf076 Free \uf076 Grant Fund \uf076 Uninsured Eligible\n\nContact Phone: 410-264-1441 Email: rebirth4hope@gmail.com Website: https://www.rebirth4hope.org/ Provider Type \uf076 Case Manager \uf076 CHOW \uf076 Community Healthcare Navigator \uf076 Peer Support Specialist\n\nHours Mon. – Fri. 9 a.m. to 4 p.m.\n\nCounties Served Caroline, Dorchester, W

In [ ]:
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [ ]:
query = "Anxiety?"

similar_docs = vectorstore.similarity_search(query)

In [ ]:
similar_docs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="GPT-4o mini",
    temperature=0.1
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)